# ``math.isclose()`` example: Newton's method

Newton's method of succesive approximations can be used to compute the square root. 

> **How Newton's method works**
>
> To compute `sqrt(n)`, the algorithm starts with a starting `guess=n/2` and a `better_guess`: the average of `guess` and `n/guess`. If those two guesses are equal or very close, the square root is the `better_guess`. If not, the `better_guess` is used as the `guess`, and a new `better_guess` is computed as the average of that and `n/guess`. This process quickly converges to a very good approximation of the square root.

The implementation below uses the `math.isclose()` function (new in Python 3.5) to test whether the `better_guess` is close to the current one, which means further approximations will not be useful and the `better_guess` is an acceptable result.

In [1]:
import math

def newton_sqrt(n, verbose=False):
    guess = n / 2
    while True:
        if verbose: print('guess ->', guess)
        better_guess = (guess + n/guess) / 2
        if math.isclose(guess, better_guess):
            return better_guess
        guess = better_guess

Sample use:

In [2]:
newton_sqrt(100)

10.0

Using `verbose=True`, we can see how quickly Newton's algorithm converges to the solution:

In [3]:
newton_sqrt(100, True)

guess -> 50.0
guess -> 26.0
guess -> 14.923076923076923
guess -> 10.812053925455988
guess -> 10.030495203889796
guess -> 10.000046356507898
guess -> 10.000000000107445


10.0

Applying `newton_sqrt` to numbers from 9 to 16, we see that some results are a little different from those from `math.sqrt`.

In [4]:
for n in range(9, 17):
    computed = newton_sqrt(n)
    expected = math.sqrt(n)
    delta = computed - expected
    print('sqrt({:2d}): {:.20f} {:.20f} {:.20f}'.format(n, computed, expected, delta))

sqrt( 9): 3.00000000000000000000 3.00000000000000000000 0.00000000000000000000
sqrt(10): 3.16227766016837907870 3.16227766016837952279 -0.00000000000000044409
sqrt(11): 3.31662479035539980998 3.31662479035539980998 0.00000000000000000000
sqrt(12): 3.46410161513775438635 3.46410161513775438635 0.00000000000000000000
sqrt(13): 3.60555127546398956895 3.60555127546398912486 0.00000000000000044409
sqrt(14): 3.74165738677394132949 3.74165738677394132949 0.00000000000000000000
sqrt(15): 3.87298334620741702139 3.87298334620741702139 0.00000000000000000000
sqrt(16): 4.00000000000000000000 4.00000000000000000000 0.00000000000000000000
